# DIY Covid-19 Dashboard

**Click the refresh data button to update the graph.**

In [1]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API
import datetime

In [2]:
%matplotlib inline
plt.rcParams['figure.dpi'] = 100

In [3]:
with open("newdata.json", "rt") as INFILE:
    data=json.load(INFILE)

In [4]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def wrangle_data(data):
    datalist=data['data']

    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()

    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])

    index=pd.date_range(startdate, enddate, freq='D')
    newdatadf=pd.DataFrame(index=index, columns=['cases', 'deaths'])

    for entry in datalist:
        date=parse_date(entry['date'])
        for column in ['cases', 'deaths']:
            if pd.isna(newdatadf.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                newdatadf.loc[date, column]=value
            
    newdatadf.fillna(0.0, inplace=True)
    return newdatadf

newdatadf=wrangle_data(data)

In [5]:
def access_api():
    filters=[
    'areaType=overview'
]
    structure={
    "date": "date",
    "cases": "newCasesByPublishDate",
    "deaths": "newDeaths28DaysByPublishDate"   
}
    api=Cov19API(filters=filters, structure=structure)
    newdata=api.get_json()
    return newdata

In [6]:
apibutton=wdg.Button(
    description='Refresh Data',
    disabled=False,
    button_style='success',
    tooltip='Click to download current Public Health England data',
    icon='refresh'
)

output1=wdg.Output()
@output1.capture(clear_output=True, wait=True)
def api_button_callback(button):
    apidata=access_api()
    global newdatadf
    newdatadf=wrangle_data(apidata)
    refresh_graph()
    apibutton.icon="check"
    api_button_callback=datetime.datetime.now()
    print(api_button_callback)

apibutton.on_click(api_button_callback)
display(apibutton)
output1

Button(button_style='success', description='Refresh Data', icon='refresh', style=ButtonStyle(), tooltip='Click…

Output()

In [7]:
def plot_newdata_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        newdatadf[list(gcols)].plot(logy=logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
        
series=wdg.SelectMultiple(
    options=['cases', 'deaths'],
    value=['cases', 'deaths'],
    rows=2,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

controls=wdg.HBox([series, scale])
graph=wdg.interactive_output(plot_newdata_graph, {'gcols': series, 'gscale': scale})
display(controls, graph)

def refresh_graph():
    current=scale.value
    if current==scale.options[0]:
        other=scale.options[1]
    else:
        other=scale.options[0]
    scale.value=other
    scale.value=current

Output()

**The data is presented on a linear scale and a logarithmic scale. The graph shows the number of confirmed Covid-19 cases and deaths per day in the United Kingdom. *Cases* represents new cases by publish date and *Deaths* represents new deaths within 28 days of a positive test. The numbers started rising in March and passed the first peak in April. The second wave started in September with the number of cases far exceeding those from the first wave.**

**Author and Copyright Notice** (C) Rafayet Tarafder, 2020 (r.tarafder@se20.qmul.ac.uk). All rights reserved. *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*